In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler,OrdinalEncoder,TargetEncoder,PowerTransformer
from sklearn.compose import ColumnTransformer,TransformedTargetRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,KFold,RandomizedSearchCV
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import optuna
from plotly.io import show
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [120]:
df = pd.read_csv('final_data.csv')
df = df.drop(['Title','Price_per_sqft','Is_in_Dhaka'],axis=1)
df.head()

,Bedrooms,Bathrooms,Floor_area,City,location_area,Price_in_Cr,floor_level
0,3.0,4.0,1960.0,dhaka,gulshan,3.90,low
1,3.0,3.0,1705.0,dhaka,kalabagan,1.69,low
2,3.0,3.0,1370.0,dhaka,dhanmondi,1.25,medium
3,3.0,3.0,2125.0,dhaka,bashundhara,2.00,medium
4,3.0,3.0,2687.0,dhaka,banani,4.75,medium


In [121]:
x = df.drop('Price_in_Cr',axis=1)
y = df['Price_in_Cr']

# For Tree Dump

In [115]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

transformer = ColumnTransformer([
    ("log_transform",PowerTransformer(method='yeo-johnson'),['Floor_area']),
    ('Onehot',OneHotEncoder(handle_unknown='ignore',sparse_output=False),['City','Bedrooms','Bathrooms','location_area']),
    ('Ordinal',OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1),['floor_level']),
    # ('Target_Encoding',TargetEncoder(),['location_area']),
    ('Scaler',StandardScaler(),['Floor_area'])
],remainder='passthrough')

model_pipe = Pipeline([
    ('preprocessing',transformer),
    ('model',XGBRegressor(random_state=0,n_jobs=1,n_estimators=1,gamma=10,min_child_weight=10))
])


# final_pipe = TransformedTargetRegressor(regressor=model_pipe,func=np.log1p,inverse_func=np.expm1)

model = model_pipe.fit(x_train,y_train)
y_pred = model.predict(x_test)

MSE = mean_squared_error(y_test,y_pred)
MAE= mean_absolute_error(y_test,y_pred)
R2 = r2_score(y_test,y_pred)

print(f'Base XGboost Model\nMSE:{MSE}\nMAE:{MAE}\nR2:{R2}')


Base XGboost Model
MSE:0.3195075624442113
MAE:0.35638607304339737
R2:0.3478744408203347


In [116]:
tree_dump = model.steps[1][1].get_booster().get_dump(dump_format='text',with_stats=True)

for trees in tree_dump:
    print(trees)

0:[f0<1.38927984] yes=1,no=2,missing=2,gain=410.353607,cover=1681
	1:[f0<0.305330157] yes=3,no=4,missing=4,gain=74.0723877,cover=1542
		3:[f0<-0.373150617] yes=7,no=8,missing=8,gain=16.0938034,cover=1106
			7:leaf=-0.123454757,cover=532
			8:leaf=-0.0509068482,cover=574
		4:[f3<1] yes=9,no=10,missing=10,gain=16.4838543,cover=436
			9:leaf=0.0171754397,cover=284
			10:leaf=0.139396116,cover=152
	2:[f0<2.24198723] yes=5,no=6,missing=6,gain=88.6400146,cover=139
		5:leaf=0.365454674,cover=111
		6:leaf=0.962734401,cover=28



# Optuna

In [133]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

floor_area_pipe = Pipeline([
    ('log_transform',PowerTransformer(method='yeo-johnson')),
    ('scalar',StandardScaler())
])

transformer = ColumnTransformer([
    ("floor_area",floor_area_pipe,['Floor_area']),
    ("log_transform",PowerTransformer(method='yeo-johnson'),['Floor_area']),
    ('Onehot',OneHotEncoder(handle_unknown='ignore',sparse_output=False),['City','Bedrooms','Bathrooms','location_area']),
    ('Ordinal',OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1),['floor_level']),
],remainder='passthrough')


def objective(trial,preprocessor):
     regressor_name = trial.suggest_categorical('regressor',['XGBRegressor'])

     if regressor_name == 'XGBRegressor':
        
        gamma = trial.suggest_int('model__gamma',0,0.5)
        reg_lambda = trial.suggest_float('model__reg_lambda',1,2)
        n_estimators = trial.suggest_int('model__n_estimators',50,300)
        learning_rate = trial.suggest_float('model__eta',0.05,0.3)
        min_child_weight = trial.suggest_int('model__min_child_weight',1,3)
        subsample = trial.suggest_float('model__subsample',0.7,1)
        colsample_bytree = trial.suggest_float('model__colsample_bytree',0.7,1)
        max_depth = trial.suggest_int('model__max_depth',3,6)
        tree_method = trial.suggest_categorical('model__tree_method',['hist'])
        max_bin = trial.suggest_int('model__max_bin',64,256)

        model_pipe = Pipeline([
        ('preprocessor',transformer),
        ('model',XGBRegressor(random_state=0,gamma=gamma,reg_lambda=reg_lambda,
                              n_estimators=n_estimators,learning_rate=learning_rate,min_child_weight=min_child_weight,
                              subsample=subsample,colsample_bytree=colsample_bytree,
                              max_depth=max_depth,tree_method=tree_method,max_bin=max_bin))
                              ])
        
        final_pipe = TransformedTargetRegressor(regressor=model_pipe,func=np.log1p,inverse_func=np.expm1)

        trial.set_user_attr('model',final_pipe)

        cv = cross_val_score(final_pipe,x_train,y_train,cv=KFold(n_splits=5,shuffle=True,random_state=0),scoring='neg_mean_squared_error')

        return cv.mean()
     

study = optuna.create_study(direction='maximize',sampler=optuna.samplers.TPESampler(seed=0),pruner=optuna.pruners.HyperbandPruner())
study.optimize(lambda trial: objective(trial,transformer),n_trials=100,n_jobs=-1)

print(f'Best neg score: {study.best_value}')
print(f'Best Params: {study.best_params}')

final_model = study.best_trial.user_attrs['model']
final_model.fit(x_train,y_train)
y_pred = final_model.predict(x_test)

print(f'MSE: {mean_squared_error(y_test,y_pred)}')
print(f'MAE: {mean_absolute_error(y_test,y_pred)}')
print(f'R2: {r2_score(y_test,y_pred)}')


[I 2025-09-26 15:48:20,797] A new study created in memory with name: no-name-2612fd50-5d57-4763-9a69-e090838f813e
[I 2025-09-26 15:48:22,426] Trial 0 finished with value: -0.07824805020647402 and parameters: {'regressor': 'XGBRegressor', 'model__gamma': 0, 'model__reg_lambda': 1.9251412884115324, 'model__n_estimators': 82, 'model__eta': 0.2126780714404331, 'model__min_child_weight': 1, 'model__subsample': 0.8785341790871162, 'model__colsample_bytree': 0.7489058879373668, 'model__max_depth': 3, 'model__tree_method': 'hist', 'model__max_bin': 211}. Best is trial 0 with value: -0.07824805020647402.
[I 2025-09-26 15:48:23,116] Trial 1 finished with value: -0.09468769832601641 and parameters: {'regressor': 'XGBRegressor', 'model__gamma': 0, 'model__reg_lambda': 1.0448176185697824, 'model__n_estimators': 86, 'model__eta': 0.1944400191664346, 'model__min_child_weight': 3, 'model__subsample': 0.9717890117871818, 'model__colsample_bytree': 0.8333299883679455, 'model__max_depth': 6, 'model__tree

Best neg score: -0.06669082139967156
Best Params: {'regressor': 'XGBRegressor', 'model__gamma': 0, 'model__reg_lambda': 1.5081004834272944, 'model__n_estimators': 248, 'model__eta': 0.12489512720368913, 'model__min_child_weight': 1, 'model__subsample': 0.8635394158144538, 'model__colsample_bytree': 0.8880124630656017, 'model__max_depth': 3, 'model__tree_method': 'hist', 'model__max_bin': 245}
MSE: 0.06478102536193789
MAE: 0.14448775030513275
R2: 0.8677797731446115
